In [24]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
print(os.getcwd())

c:\Users\mhu\Documents\github\topic_model_it


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Pfad zur Rohdatei
src = r"data/informatikkurse.csv"
if not os.path.exists(src):
    raise FileNotFoundError(f"{src} nicht gefunden")

# CSV laden - JETZT MIT sep=';'
informatik_df = pd.read_csv(src, sep=';')

# Train/Test Split
training_set, test_set = train_test_split(informatik_df, test_size=0.2, random_state=42, shuffle=True)

# Ordner erstellen und speichern
os.makedirs("data/processed", exist_ok=True)
training_set.to_csv("data/processed/train_data.csv", index=False)
test_set.to_csv("data/processed/test_data.csv", index=False)

C:\Users\mhu\AppData\Local\Temp\ipykernel_27128\2363639698.py:11: DtypeWarning: Columns (5,6,10,11,14,15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  informatik_df = pd.read_csv(src, sep=';')


In [26]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

In [27]:
# Training-Datensatz
training_set = pd.read_csv("data/processed/train_data.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

In [28]:
# Test-Datensatz
test_data = pd.read_csv("data/processed/test_data.csv")
test_set = test_data.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
test_set['titel_kursbesch'] = test_set['veranstaltung_titel'] + ' ' + test_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
test_docs = test_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

## Stopwords

In [29]:
import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)

## Model

Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

In [30]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

In [31]:

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


In [32]:
# BERTopic initialisieren
topic_model = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  nr_topics=25, 
  language="multilingual",
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 15,
  representation_model=representation_model
)

Training

In [33]:
topic_model_quanten = topic_model.fit(docs)

In [34]:
# Korrekt & knapp
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

 
client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(client=client, model="gpt-4o-mini", delay_in_seconds=5)

# nutze dieselben Texte wie beim Fit (z. B. `docs`)
topic_model.update_topics(docs, representation_model=rep)
topic_model.get_topic_info()

 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7145,-1_Computer Science Seminars,[Computer Science Seminars],[Introduction to Modern Cryptography Cryptogra...
1,0,20296,0_Scientific work in Computer Science,[Scientific work in Computer Science],"[Grundlagen der Informatik I , Informatik-Koll..."
2,1,2582,1_Robotics and AI Education,[Robotics and AI Education],"[Machine Learning , Machine Learning , Machine..."
3,2,2265,2_Medical Informatics Introduction,[Medical Informatics Introduction],"[Querschnittsbereich Epidemiologie, Medizinisc..."
4,3,2148,3_Data Science Education,[Data Science Education],"[Übung zu: Data Science , Einführung in Data S..."
5,4,1883,4_3D Computer Vision,[3D Computer Vision],"[Computer Vision , Übungen zu Computer Vision..."
6,5,1263,5_Post-Quantum Cryptography,[Post-Quantum Cryptography],"[IT-Security Projekt , Information Security , ..."
7,6,1063,6_Digital Transformation Seminar,[Digital Transformation Seminar],[Master-Seminar - Digital Transformation (IN21...
8,7,808,7_Digital Transformation and Sustainability,[Digital Transformation and Sustainability],[Master-Seminar - Digital Transformation & Sus...
9,8,291,8_Linear Algebra Structures,[Linear Algebra Structures],"[Lineare Algebra und Diskrete Strukturen 1 , ..."


In [36]:
# # BERTopic auf Test-Daten anwenden
topics, probs = topic_model_quanten.transform(test_docs)

In [ ]:
topic_model_quanten.get_topic_info()

In [ ]:
# Outlier reduzieren
topics = topic_model_quanten.reduce_outliers(test_docs, topics)

In [ ]:
topics

In [ ]:
# Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

In [ ]:
row_number = 0
metric = 0
while row_number < len(ground_truth):
  # Den Goldstandard in eine Liste von Keywords umwandeln
  ground_truth_current_iteration = ground_truth[row_number].split(", ")
  result_current_iteration = dataframe_with_results.at[row_number, "Representation"]
  # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
  if any(element in result_current_iteration for element in ground_truth_current_iteration):
      metric += 1
  else: 
      metric += 0
      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("--------------------------------------------------------------------------------------------------")
  row_number = row_number+1

metric_score = metric/row_number
print(metric_score)